In [7]:
import os
import numpy as np
import pandas as pd

from hmog import HmogHelper
from tqdm import tqdm

dataset_path = 'preprocessed_data/'

In [12]:
df = pd.read_csv(os.path.join(dataset_path, 'hmog_dataset_unified_df_ids_types_indices.csv'), index_col=0)
df_20s = pd.read_csv(os.path.join(dataset_path, 'hmog_dataset_unified_df_20s.csv'), index_col=0)
df_40s = pd.read_csv(os.path.join(dataset_path, 'hmog_dataset_unified_df_40s.csv'), index_col=0)
df_60s = pd.read_csv(os.path.join(dataset_path, 'hmog_dataset_unified_df_60s.csv'), index_col=0)
df_80s = pd.read_csv(os.path.join(dataset_path, 'hmog_dataset_unified_df_80s.csv'), index_col=0)
df_100s = pd.read_csv(os.path.join(dataset_path, 'hmog_dataset_unified_df_100s.csv'), index_col=0)
df_120s = pd.read_csv(os.path.join(dataset_path, 'hmog_dataset_unified_df_120s.csv'), index_col=0)
df_140s = pd.read_csv(os.path.join(dataset_path, 'hmog_dataset_unified_df_140s.csv'), index_col=0)

In [9]:
subjects_list = list(df['user_id'].unique())

In [13]:
# Let's scale the data

from sklearn.preprocessing import StandardScaler

hmog_features = list(map(str, range(1, 65)))

df_scaled = df.copy()
scaler = StandardScaler()

df_scaled[hmog_features] = scaler.fit_transform(df[hmog_features])
df_20s[hmog_features] = scaler.transform(df_20s[hmog_features])
df_40s[hmog_features] = scaler.transform(df_40s[hmog_features])
df_60s[hmog_features] = scaler.transform(df_60s[hmog_features])
df_80s[hmog_features] = scaler.transform(df_80s[hmog_features])
df_100s[hmog_features] = scaler.transform(df_100s[hmog_features])
df_120s[hmog_features] = scaler.transform(df_120s[hmog_features])
df_140s[hmog_features] = scaler.transform(df_140s[hmog_features])

df = df_scaled

In [14]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

def create_model():
    model = Sequential()
    model.add(Dense(60, input_shape=(64,), activation='relu'))
    model.add(Dense(1, activation='sigmoid'))

    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [33]:
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split

WINDOW_SIZES = [20000, 40000, 60000, 80000, 100000, 120000, 140000]

session_key_to_int = {'s': 0, 'w': 1}

def test_eer(train_key: str, test_key: str):
    window_eers = {}

    for window_size in WINDOW_SIZES:
        eer_list = []

        for i in tqdm(range(len(subjects_list))):
            subject = subjects_list[i]
            
            train_hmog_vector_positive = df[( (df['user_id'] == subject) & (df['session_type'] == session_key_to_int[train_key]) & ( (df['session_index'] == 0) | (df['session_index'] == 1) ) )]
            
            test_hmog_vector_positive = df[( (df['user_id'] == subject) & (df['session_type'] == session_key_to_int[test_key]) & (( df['session_index'] == 2) | (df['session_index'] == 3) ) )]
            
            test_hmog_vector_positive_np = test_hmog_vector_positive[hmog_features].to_numpy()
            
            another_subject_i = i + 1 if i < len(subjects_list) else 0
            another_subject = subjects_list[another_subject_i]

            train_hmog_vector_negative = df[( (df['user_id'] == another_subject) & (df['session_type'] == session_key_to_int[train_key]) & ( (df['session_index'] == 0) | (df['session_index'] == 1) ) )]
            
            data = pd.concat([train_hmog_vector_positive, train_hmog_vector_negative], ignore_index=True)
            data[data['user_id'] == subject] = 1
            data[data['user_id'] == another_subject] = 0
            
            X, y = data[hmog_features], data['user_id']
            # print(X.shape, y.shape)
            # print(y.unique())
            
            model = create_model()
            model.fit(X, y, epochs=2, verbose=1)
            
            test_hmog_vectors_all_users = df_20s[( ~(df_20s['user_id'] == subject) & (df_20s['session_type'] == session_key_to_int[test_key]) & (( df_20s['session_index'] == 2) | (df_20s['session_index'] == 3) ) )]
            test_hmog_vectors_all_users_np = test_hmog_vectors_all_users[hmog_features].to_numpy()
            
            predictions_self = model.predict(test_hmog_vector_positive_np)
            predictions_others = model.predict(test_hmog_vectors_all_users_np)
            
            scores = np.concatenate([predictions_self, predictions_others])
            
            y_true = [1] * len(predictions_self)
            y_true += ([0] * len(predictions_others))
            
            fpr, tpr, thresholds = roc_curve(y_true, scores)

            # Find the threshold where FPR equals FNR
            eer_index = np.argmin(np.abs(fpr - (1 - tpr)))
            eer = (fpr[eer_index] + (1 - tpr[eer_index])) / 2

            # Append EER to the list
            eer_list.append(eer)
                
        population_eer = np.mean(eer_list)
        window_eers[window_size] = population_eer
        print(f'Window size: {window_size / 1000}s\nPopulation EER: {window_eers[window_size]}')

    print('Done')
    return window_eers

window_eers = test_eer('w', 'w')

  0%|          | 0/92 [00:00<?, ?it/s]

Epoch 1/2
76/76 [==============================] - 0s 526us/step - loss: 0.3955 - accuracy: 0.9537
Epoch 2/2
3945/3945 [==============================] - 1s 277us/step


  1%|          | 1/92 [00:02<03:15,  2.15s/it]

Epoch 1/2
94/94 [==============================] - 0s 511us/step - loss: 0.3828 - accuracy: 0.9839
Epoch 2/2
3942/3942 [==============================] - 1s 270us/step


  2%|▏         | 2/92 [00:04<02:58,  1.99s/it]

...

Epoch 1/2
97/97 [==============================] - 0s 523us/step - loss: 0.3369 - accuracy: 0.9887
Epoch 2/2
3928/3928 [==============================] - 1s 260us/step


 98%|█████████▊| 90/92 [02:45<00:03,  1.84s/it]

Epoch 1/2
90/90 [==============================] - 0s 546us/step - loss: 0.3452 - accuracy: 0.9679
Epoch 2/2
3939/3939 [==============================] - 1s 278us/step


 99%|█████████▉| 91/92 [02:47<00:01,  1.84s/it]


IndexError: list index out of range

In [22]:
print(eer_list)

NameError: name 'eer_list' is not defined

NameError: name 'eer_list' is not defined